[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/siddmodi/Python-Task/blob/main/Python.ipynb)

In [135]:
# Importing required libraries

from bs4 import BeautifulSoup  
import pandas as pd
import requests
import openpyxl
import lxml
import os
import time

In [136]:
# Creating a pandas dataframe of Amazon Scraping file. 
df = pd.read_excel('D:\Credicxo\Amazon Scraping.xlsx',index_col=0)
df.drop('id', axis=1, inplace=True)
df

,Asin,country
0,1015,de
1,1015,fr
2,000004458X,de
3,000004458X,fr
4,1002198,de
...,...,...
995,4484924,fr
996,4485742,de
997,4485742,fr
998,4486072,de


In [137]:
# Creating a list of all the url's.

url = 'https://www.amazon.{country}/dp/{asin}'
url_list=[]
for i in range(len(df)):
    asin = df['Asin'][i]
    country = df['country'][i]
    link = f'https://www.amazon.{country}/dp/{asin}'
    url_list.append(link)
url_list

['https://www.amazon.de/dp/1015',
 'https://www.amazon.fr/dp/1015',
 'https://www.amazon.de/dp/000004458X',
 'https://www.amazon.fr/dp/000004458X',
 'https://www.amazon.de/dp/1002198',
 'https://www.amazon.fr/dp/1002198',
 'https://www.amazon.fr/dp/1002791',
 'https://www.amazon.it/dp/1002791',
 'https://www.amazon.de/dp/1002864',
 'https://www.amazon.fr/dp/1002864',
 'https://www.amazon.de/dp/1003704',
 'https://www.amazon.fr/dp/1003704',
 'https://www.amazon.de/dp/1003763',
 'https://www.amazon.fr/dp/1003763',
 'https://www.amazon.fr/dp/1004271',
 'https://www.amazon.it/dp/1004271',
 'https://www.amazon.de/dp/000101742X',
 'https://www.amazon.fr/dp/000101742X',
 'https://www.amazon.de/dp/1017519',
 'https://www.amazon.fr/dp/1017519',
 'https://www.amazon.de/dp/000102163X',
 'https://www.amazon.fr/dp/000102163X',
 'https://www.amazon.fr/dp/1022369',
 'https://www.amazon.it/dp/1022369',
 'https://www.amazon.fr/dp/1022857',
 'https://www.amazon.it/dp/1022857',
 'https://www.amazon.de/dp

In [139]:
'''Loop returning a list of dictionaries of each working URL 4 key-value pairs and returning URL not available
   when URL throws Error 404. Also calculate the time taken in this process.'''

start = time.time()
l = []
for i in url_list:
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'} 
    try:
        webpage = requests.get(url =i ,headers=headers).text
        soup = BeautifulSoup(webpage,"html.parser")
        # title
        try:
            title = soup.find_all('h1',id='title')[0].text.strip()
        except Exception:
            print(i,' URL not available')
            continue
        # img 
        try:
            img = soup.find_all('div',id='img-canvas')[0].find_all('img')[0]['src']
        except Exception:
            img = '---'
        # price
        try:
            price = soup.find_all('a',href='javascript:void(0)')[1].find_all('span')[1].text.strip()
        except Exception:
            price = '---'
        # details
        details = ''
        try:
            for i in range(5):
                try:
                    s = soup.find_all('div',id='detailBullets_feature_div')[0].find_all('ul')[0].find_all('li')[i].find_all('span')[0].find_all('span')[1].text
                    details = details+' '+s
                except IndexError as e:
                    break
        except Exception:
            details = '---'
        # dictionary
        d = {}
        d['product_title']   = title
        d['product_img_url'] = img
        d['product_price']   = price
        d['product_details'] = details
        l.append(d)
    except Exception as e:
        print(i,' URL not available')
end = time.time()
elapsed_time = end-start

In [143]:
elapsed_time

594.1293199062347

In [144]:
# Jsonify the resultant data.
import json
jsonStr = json.dumps(l)
print(jsonStr)

# Writing to sample.json
with open("scrap.json", "w") as outfile:
    outfile.write(jsonStr)

[{"product_title": "Old Spice Rasur Creme - 70 G (original) - Packung Von 2", "product_img_url": "---", "product_price": "---", "product_details": ""}, {"product_title": "Short Story: Violoncello und Klavier. (Cello-Bibliothek)   Musiknoten \u2013 6. Februar 1986", "product_img_url": "https://images-eu.ssl-images-amazon.com/images/I/41zw0783tpL._SX198_BO1,204,203,200_QL40_ML2_.jpg", "product_price": "11,50\u00a0\u20ac", "product_details": " SCHOTT MUSIC GmbH & Co KG, Mainz (6. Februar 1986) Franz\u00f6sisch 10 Seiten 000101742X 978-0001017429"}, {"product_title": "Short story (dushkin/hegyi) violon   Partition \u2013 1 janvier 2000", "product_img_url": "https://images-eu.ssl-images-amazon.com/images/I/41zw0783tpL._SX218_BO1,204,203,200_QL40_ML2_.jpg", "product_price": "\u00e0 partir de 23,72\u00a0\u20ac", "product_details": " SCHOTT (1 janvier 2000) Fran\u00e7ais 000101742X 978-0001017429 601 g"}, {"product_title": "Concerto grosso g-Moll: Weihnachtskonzert. op. 6/8. 2 Violinen, Violon

# Working with mysql database

In [145]:
# Creating a list of tuples. Each tuple consisting data of a single product.

l2=[]
for i in l:
    l1=[]
    l1.append(i['product_title'])
    l1.append(i['product_img_url'])
    l1.append(i['product_price'])
    l1.append(i['product_details'])
    t = tuple(l1)
    l2.append(t)

In [147]:
import pymysql
import mysql.connector

mydb = mysql.connector.connect(host='localhost',
                              user='root',
                              passwd='root12345',
                              database='amazon')
mycursor = mydb.cursor()

In [155]:
# Drop table if it already exist using execute() method.
mycursor.execute("DROP TABLE IF EXISTS SCRAP ")

# Create table as per requirement
sql = """CREATE TABLE SCRAP (
    PRODUCT_TITLE VARCHAR(200),
    PRODUCT_IMAGE_URL VARCHAR(100),
    PRODUCT_PRICE VARCHAR(30),
    PRODUCT_DETAILS VARCHAR(200)
   )"""

mycursor.execute(sql)

In [156]:
# Insert rows in table

mySql_insert_query = """INSERT INTO SCRAP (PRODUCT_TITLE, PRODUCT_IMAGE_URL, PRODUCT_PRICE, PRODUCT_DETAILS) 
                                VALUES (%s, %s, %s, %s) """

records_to_insert = l2

mycursor = mydb.cursor()
mycursor.executemany(mySql_insert_query, records_to_insert)
mydb.commit()
print(mycursor.rowcount, "Record inserted successfully into SCRAP table")
mydb.close()

41 Record inserted successfully into SCRAP table
